In [8]:
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [9]:
def convert_from_json_to_dataframe(file_path, record_path=['data', 'paragraphs', 'qas', 'answers']):
    file = json.loads(open(file_path).read())
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file, record_path[:-2])
    index = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = index
    data = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
    data['context_id'] = data['context'].factorize()[0]
    return data
    

train_file_path = 'train-v1.1.json'
train_data = convert_from_json_to_dataframe(train_file_path)
train_data

,id,question,context,answers,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 188, 'text': 'a copper statue of Christ'}]",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 279, 'text': 'the Main Building'}]",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}]",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre Dame?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, Franc

In [10]:
train_data.columns

Index(['id', 'question', 'context', 'answers', 'context_id'], dtype='object')

In [11]:
train_data['answers']

0                     [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]
1                      [{'answer_start': 188, 'text': 'a copper statue of Christ'}]
2                              [{'answer_start': 279, 'text': 'the Main Building'}]
3        [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}]
4              [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}]
                                            ...                                    
87594                                     [{'answer_start': 229, 'text': 'Oregon'}]
87595                                    [{'answer_start': 414, 'text': 'Rangoon'}]
87596                                      [{'answer_start': 476, 'text': 'Minsk'}]
87597                                       [{'answer_start': 199, 'text': '1975'}]
87598                  [{'answer_start': 0, 'text': 'Kathmandu Metropolitan City'}]
Name: answers, Length: 87599, dtype: object

In [19]:
train_data[['answer_start', 'answer_text']] = train_data['answers'].apply(lambda x: pd.Series([x[0]['answer_start'], x[0]['text']]))
train_data.drop('answers', axis=1, inplace=True)

In [21]:
train_data.drop_duplicates(subset=['question'], inplace=True)
train_data.drop_duplicates(subset=['context'], inplace=True)
train_data.drop_duplicates(subset=['answer_start'], inplace=True)
train_data.drop_duplicates(subset=['answer_text'], inplace=True)
train_data.reset_index(drop=True, inplace=True)
train_data

,id,question,context,context_id,answer_start,answer_text
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",0,515,Saint Bernadette Soubirous
1,5733bf84d058e614000b61be,When did the Scholastic Magazine of Notre dame begin publishing?,"As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a liberal newspaper, Common Sense was published. Likewise, in 2003, when other students believed that the paper showed a liberal bias, the conservative paper Irish Rover went into production. Neither paper is published as often as The Observer; however, all three are distributed to all students. Finally, in Spring 2008 an undergraduate journal for political science research, Beyond Politics, made its debut.",1,248,September 1876
2,5733bed24776f41900661188,Where is the headquarters of the Congregation of the Holy Cross?,"The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.",2,119,Rome
3,5733a6424776f41900660f51,How many BS level degrees are offered in the College of Engineering at Notre Dame?,"The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master 

In [27]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

ps = PorterStemmer()
wnl = WordNetLemmatizer()

def stem_and_lemmatize(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [ps.stem(token) for token in tokens]
    lemmatized_tokens = [wnl.lemmatize(token) for token in stemmed_tokens]
    return ' '.join(lemmatized_tokens)

train_data['question'] = train_data['question'].apply(stem_and_lemmatize)
train_data['context'] = train_data['context'].apply(stem_and_lemmatize)
train_data['answer_text'] = train_data['answer_text'].apply(stem_and_lemmatize)

[nltk_data] Downloading package punkt to /home/harsh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/harsh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
train_data

,id,question,context,context_id,answer_start,answer_text
0,5733be284776f41900661182,to whom did the virgin mari allegedli appear in 1858 in lourd franc ?,"architectur , the school ha a cathol charact . atop the main build 's gold dome is a golden statu of the virgin mari . immedi in front of the main build and face it , is a copper statu of christ with arm uprai with the legend `` venit ad me omn `` . next to the main build is the basilica of the sacr heart . immedi behind the basilica is the grotto , a marian place of prayer and reflect . it is a replica of the grotto at lourd , franc where the virgin mari reputedli appear to saint bernadett soubir in 1858 . at the end of the main drive ( and in a direct line that connect through 3 statu and the gold dome ) , is a simpl , modern stone statu of mari .",0,515,saint bernadett soubir
1,5733bf84d058e614000b61be,when did the scholast magazin of notr dame begin publish ?,"a at most other univ , notr dame 's student run a number of news medium outlet . the nine student-run outlet includ three newspap , both a radio and televi station , and sever magazin and journal . begun a a one-pag journal in septemb 1876 , the scholast magazin is issu twice monthli and claim to be the oldest continu collegi public in the unit state . the other magazin , the juggler , is relea twice a year and focu on student literatur and artwork . the dome yearbook is publish annual . the newspap have vari public interest , with the observ publish daili and mainli report univ and other news , and staf by student from both notr dame and saint mari 's colleg . unlik scholast and the dome , the observ is an independ public and doe not have a faculti advisor or ani editori oversight from the univ . in 1987 , when some student believ that the observ began to show a conserv bia , a liber newspap , common sen wa publish . likewi , in 2003 , when other student believ that the paper show a liber bia , the conserv paper irish rover went into product . neither paper is publish a often a the observ ; howev , all three are distribut to all student . final , in spring 2008 an undergradu journal for polit scienc research , beyond polit , made it debut .",1,248,septemb 1876
2,5733bed24776f41900661188,where is the headquart of the congreg of the holi cross ?,"the univ is the major seat of the congreg of holi cross ( albeit not it offici headquart , which are in rome ) . it main seminari , moreau seminari , is locat on the campu across st. joseph lake from the main build . old colleg , the oldest build on campu and locat near the shore of st. mari lake , hou undergradu seminarian . retir priest and brother resid in fatima hou ( a former retreat center ) , holi cross hou , a well a columba hall near the grotto . the univ through the moreau seminari ha tie to theologian frederick buechner . while not cathol , buechner ha prai writer from notr dame and moreau seminari creat a buechner prize for preach .",2,119,rome
3,5733a6424776f41900660f51,how mani b level degr are offer in the colleg of engin at notr dame ?,"the colleg of engin wa establish in 1920 , howev , earli cour in civil and mechan engin were a part of the colleg of scienc sinc the 1870 . today the colleg , hou in the fitzpatrick , cush , and stinson-remick hall of engin , includ five depart of studi – aerospac and mechan engin , chemic and biomolecular engin , civil engin and geolog scienc , comput scienc and engin , and electr engin – with eight b. . degr offer . addit , the colleg offer five-year dual degr program with the colleg of art and letter and of busi award addit b.a . and master of busi administr ( mba ) degr , respect .",3,487,eight
4,5733a70c4776f41900660f64,what entiti provid help with the manag of time for new student at notr dame ?,"all of notr dame 's undergradu student are a part of one of the five undergradu colleg at the school or are in the first year of studi program . the first year of studi program wa establish in 1962 to guid incom freshman in the

In [30]:
from nltk import pos_tag

def pos_tagging(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    return pos_tags

train_data['question_pos'] = train_data['question'].apply(pos_tagging)
train_data['context_pos'] = train_data['context'].apply(pos_tagging)
train_data['answer_text_pos'] = train_data['answer_text'].apply(pos_tagging)

In [31]:
train_data

,id,question,context,context_id,answer_start,answer_text,question_pos,context_pos,answer_text_pos
0,5733be284776f41900661182,to whom did the virgin mari allegedli appear in 1858 in lourd franc ?,"architectur , the school ha a cathol charact . atop the main build 's gold dome is a golden statu of the virgin mari . immedi in front of the main build and face it , is a copper statu of christ with arm uprai with the legend `` venit ad me omn `` . next to the main build is the basilica of the sacr heart . immedi behind the basilica is the grotto , a marian place of prayer and reflect . it is a replica of the grotto at lourd , franc where the virgin mari reputedli appear to saint bernadett soubir in 1858 . at the end of the main drive ( and in a direct line that connect through 3 statu and the gold dome ) , is a simpl , modern stone statu of mari .",0,515,saint bernadett soubir,"[(to, TO), (whom, WP), (did, VBD), (the, DT), (virgin, NN), (mari, NN), (allegedli, NN), (appear, VBP), (in, IN), (1858, CD), (in, IN), (lourd, JJ), (franc, NN), (?, .)]","[(architectur, NN), (,, ,), (the, DT), (school, NN), (ha, VBD), (a, DT), (cathol, NN), (charact, NN), (., .), (atop, VB), (the, DT), (main, JJ), (build, NN), ('s, POS), (gold, NN), (dome, NN), (is, VBZ), (a, DT), (golden, JJ), (statu, NN), (of, IN), (the, DT), (virgin, NN), (mari, NN), (., .), (immedi, NN), (in, IN), (front, NN), (of, IN), (the, DT), (main, JJ), (build, NN), (and, CC), (face, NN), (it, PRP), (,, ,), (is, VBZ), (a, DT), (copper, NN), (statu, NN), (of, IN), (christ, NN), (with, IN), (arm, NN), (uprai, NN), (with, IN), (the, DT), (legend, NN), (``, ``), (venit, NN), (ad, NN), (me, PRP), (omn, IN), (``, ``), (., .), (next, JJ), (to, TO), (the, DT), (main, JJ), (build, NN), (is, VBZ), (the, DT), (basilica, NN), (of, IN), (the, DT), (sacr, JJ), (heart, NN), (., .), (immedi, NN), (behind, IN), (the, DT), (basilica, NN), (is, VBZ), (the, DT), (grotto, NN), (,, ,), (a, DT), (marian, JJ), (place, NN), (of, IN), (prayer, NN), (and, CC), (reflect, NN), (., .), (it, PRP), (is, VBZ), (a, DT), (replica, NN), (of, IN), (the, DT), (grotto, NN), (at, IN), (lourd, NN), (,, ,), (franc, NN), (where, WRB), (the, DT), (virgin, NN), (mari, NN), (reputedli, NN), ...]","[(saint, NN), (bernadett, NN), (soubir, NN)]"
1,5733bf84d058e614000b61be,when did the scholast magazin of notr dame begin publish ?,"a at most other univ , notr dame 's student run a number of news medium outlet . the nine student-run outlet includ three newspap , both a radio and televi station , and sever magazin and journal . begun a a one-pag journal in septemb 1876 , the scholast magazin is issu twice monthli and claim to be the oldest continu collegi public in the unit state . the other magazin , the juggler , is relea twice a year and focu on student literatur and artwork . the dome yearbook is publish annual . the newspap have vari public interest , with the observ publish daili and mainli report univ and other news , and staf by student from both notr dame and saint mari 's colleg . unlik scholast and the dome , the observ is an independ public and doe not have a faculti advisor or ani editori oversight from the univ . in 1987 , when some student believ that the observ began to show a conserv bia , a liber newspap , common sen wa publish . likewi , in 2003 , when other student believ that the paper show a liber bia , the conserv paper irish rover went into product . neither paper is publish a often a the observ ; howev , all three are distribut to all student . final , in spring 2008 an undergradu journal for polit scienc research , beyond polit , made it debut .",1,248,septemb 1876,"[(when, WRB), (did, VBD), (the, DT), (scholast, NN), (magazin, NN), (of, IN), (notr, JJ), (dame, NN), (begin, VB), (publish, JJ), (?, .)]","[(a, DT), (at, IN), (most, JJS), (other, JJ), (univ, NN), (,, ,), (notr, JJ), (dame, NN), ('s, POS), (student, NN), (run, VB), (a, DT), (number, NN), (of, IN), (news, NN), (medium, NN), (outlet, NN), (., .),